<img src='img/logo.png'>
<img src='img/title.png'>

# Unsupervised Learning and Feature Extraction

# Table of Contents
* [Unsupervised Learning and Feature Extraction](#Unsupervised-Learning-and-Feature-Extraction)
	* [Dimensionality Reduction, Feature Extraction and Manifold Learning](#Dimensionality-Reduction,-Feature-Extraction-and-Manifold-Learning)
		* [Principal Component Analysis (PCA)](#Principal-Component-Analysis-%28PCA%29)
			* [Applying PCA to the cancer dataset for visualization](#Applying-PCA-to-the-cancer-dataset-for-visualization)
		* [`PCA`: rotation, *sphere*ing,  and whitening example](#PCA:-rotation,-*sphere*ing,--and-whitening-example)
			* [Center the data](#Center-the-data)
			* [PCA Take 1: Rotation from Pricipal (Data) Axes to Standard Axes](#PCA-Take-1:-Rotation-from-Pricipal-%28Data%29-Axes-to-Standard-Axes)
			* [PCA Take 2:  *Sphere*ing (i.e., Whitening or IID Whitening)](#PCA-Take-2:--*Sphere*ing-%28i.e.,-Whitening-or-IID-Whitening%29)
			* [PCA Take 3:  On Standardized Data](#PCA-Take-3:--On-Standardized-Data)
		* [Another PCA Example](#Another-PCA-Example)
		* [Non-Negative Matrix Factorization (NMF)](#Non-Negative-Matrix-Factorization-%28NMF%29)
			* [Applying NMF to synthetic data](#Applying-NMF-to-synthetic-data)
		* [Manifold learning with t-SNE](#Manifold-learning-with-t-SNE)
	* [Independent Component Analysis](#Independent-Component-Analysis)
		* [Using a `t` distribution with small degrees of freedom (non-Gaussian)](#Using-a-t-distribution-with-small-degrees-of-freedom-%28non-Gaussian%29)
* [Summary](#Summary)


In [ ]:
#Fix working directory
%cd notebooks

In [ ]:
import numpy as np
import pandas as pd

import holoviews as hv
import hvplot.pandas
import panel as pn
hv.extension('bokeh')

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['image.interpolation'] = "none"
np.set_printoptions(precision=3)
plt.rcParams['image.cmap'] = "gray"

import src.mglearn as mglearn

## Dimensionality Reduction, Feature Extraction and Manifold Learning

Let's examine the breast cancer data set. There are many features with varying ranges of values.

* What subset are most useful?
* Which ones exbhibit colinearity?

These can be hard questions to answer. It would be better to reduce the dimensionality of the features.

In [ ]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target
df['target'] = df['target'].replace({0:'malignant', 1:'benign'})
df.head()

In [ ]:
malignant = df['target'] == 'malignant'
benign = df['target'] == 'benign'

fig, axes = plt.subplots(15, 2, figsize=(12, 22))
for c,ax in zip(df.columns.drop('target'), axes.ravel()):
    opts = dict(bins=50, alpha=0.5, ax=ax, title=c, yticks=[], density=True)
    
    df.loc[malignant, c].plot.hist(color='r', **opts)
    df.loc[benign, c].plot.hist(color='g', **opts)
    
fig.tight_layout()

### Principal Component Analysis (PCA)

Methods like PCA can simplify a colinear input matrix to a smaller set of representative columns (axes).

PCA finds orthogonal axes that best explain variance in the input matrix. Scikit-learn has two PCA models, [sklearn.decomposition.PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) and [sklearn.decomposition.IncrementalPCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.IncrementalPCA.html), with `IncrementalPCA` allowing fitting to large input matrices that are subsampled.

In [ ]:
mglearn.plots.plot_pca_illustration()
plt.suptitle("pca_illustration");

#### Applying PCA to the cancer dataset for visualization

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

cancer = load_breast_cancer()

scaler = StandardScaler()
scaler.fit(cancer.data)
X_scaled = scaler.transform(cancer.data)

In [ ]:
from sklearn.decomposition import PCA

# keep the first two principal components of the data
pca = PCA(n_components=2)

# fit PCA model to beast cancer data
pca.fit(X_scaled)
# transform data onto the first two principal components
X_pca = pca.transform(X_scaled)

print("Original shape: %s" % str(X_scaled.shape))
print("Reduced shape: %s" % str(X_pca.shape))

In [ ]:
p = (hv.Points((X_pca[:,0], X_pca[:,1],cancer.target),
               kdims=['first component', 'second component'], vdims=['Category'])
     .options(color='Category', cmap='Category10', size=10, alpha=0.6, width=800, height=600)
    )
p

In [ ]:
pca.components_.shape

In [ ]:
print(pca.components_)

PCA components are linear combinations of the original features. Each column is represented in both components.

* Red: more weight in the first component
* Blue: more weight in the second component

In [ ]:
import colorcet

plt.matshow(pca.components_, cmap='coolwarm')
plt.yticks([0, 1], ["first component", "second component"])
plt.colorbar()
plt.xticks(range(len(cancer.feature_names)),
           cancer.feature_names, rotation=60, ha='left');

### `PCA`: rotation, *sphere*ing,  and whitening example

The following series of cells show how preprocessing steps such as subtracting off the mean or calculating a z-score `(X - X.mean() / X.std())` can improve the fit of a PCA model.

In [ ]:
plot_options = dict(size=8, width=800, height=400, padding=0.1)

In [ ]:
mean = [8,12]
cov = [[2,3],[5,8]]
data = np.random.multivariate_normal(mean, cov, 100)

original = hv.Points(data, label='original').options(**plot_options)
original

#### Center the data

In [ ]:
centered = data - data.mean(axis=0)

center = hv.Points(centered, label='centered').options(**plot_options, legend_position='bottom_right')
p = original * center
p

#### PCA Take 1: Rotation from Pricipal (Data) Axes to Standard Axes

In [ ]:
pca = PCA(n_components=2)

pca.fit(centered)
rotated = pca.fit_transform(centered)

rot = hv.Points(rotated, label='rotated').options(**plot_options)
p = p * rot
p

#### PCA Take 2:  *Sphere*ing (i.e., Whitening or IID Whitening)

In [ ]:
pca = PCA(n_components=2, whiten=True)

pca.fit(centered)
sphered = pca.fit_transform(centered)

white = hv.Points(sphered, label='whitened').options(**plot_options)
p = p * white
p

#### PCA Take 3:  On Standardized Data

In [ ]:
zdata = (data-np.mean(data, axis=0)) / np.std(data, axis=0, ddof=1)

z = hv.Points(zdata, label='Z-scored').options(**plot_options)
p = p * z
p

In [ ]:
pca = PCA(n_components=2)

pca.fit(zdata)
rotated_z = pca.fit_transform(zdata)

final = hv.Points(rotated_z, label='final PCA').options(**plot_options)
p = p * final
p

### Another PCA Example

A Scatter Matrix is a good way to look at matrices with colinear variables.  Here the `scatter_matrix` shows 3 of 4 columns of the feature matrix are colinear and thus a good candidate for dimensionality reduction.

In [ ]:
mean = [8,12]
cov = [[2,3],[5,8]]
X, Y = np.random.multivariate_normal(mean, cov, 100).T
expanded_data = np.c_[2*X + Y, .5*X+.5*Y, X**2 + Y, np.sqrt(X) + np.sin(Y)]
centered = expanded_data - np.mean(expanded_data, axis=0)

In [ ]:
df = pd.DataFrame(centered, columns=[str(i) for i in range(4)])
hvplot.scatter_matrix(df)

In [ ]:
pca = PCA(n_components=2, whiten=True)
pca.fit(centered)

fmt = "Component %d explains %f of variance"
print("\n".join(fmt % (idx, evr) for idx, evr in enumerate(pca.explained_variance_ratio_)))

sphered = pca.fit_transform(centered)
hv.Points(sphered, label='PCA').options(**plot_options)

### Non-Negative Matrix Factorization (NMF)

NMF is a decomposition method solving the following matrix algebra equality on 3 positive matrices:
```python
V = W * H
```

where:
 * `V`'s shape is `(n, m)`, 
 * `W`'s shape is `(n, n_components)`, and 
 * `H`'s shape is `(n_components, m)`.

[This SIAM conference presentation](https://www.siam.org/meetings/sdm11/park.pdf) provides more background on NMF and comparison to other decomposition methods.

NMF with scikit-learn [is described here sklearn.decomposition.NMF](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html)

#### Applying NMF to synthetic data

In [ ]:
mglearn.plots.plot_nmf_illustration()
plt.suptitle("nmf_illustration");

### Manifold learning with t-SNE

The following cells show manifold learning with a better fit to the `digits` scikit-learn example data than PCA.  More information on manifold learning can be found in 
* [this comparison of manifold methods](http://scikit-learn.org/stable/auto_examples/manifold/plot_compare_methods.html)
* [a manifold example with severed sphere](http://scikit-learn.org/stable/auto_examples/manifold/plot_manifold_sphere.html#sphx-glr-auto-examples-manifold-plot-manifold-sphere-py)

Note [the full documentation on the manifold module](http://scikit-learn.org/stable/modules/manifold.html#manifold) describes a variety of useful estimators we do not cover in this notebook, such as [LocallyLinearEmbedding](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.LocallyLinearEmbedding.html) and [SpectralEmbedding](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.SpectralEmbedding.html).

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

fig, axes = plt.subplots(2, 5, figsize=(10, 5),
                         subplot_kw={'xticks':(), 'yticks': ()})
for ax, img in zip(axes.ravel(), digits.images):
    ax.imshow(img)

In [ ]:
# build a PCA model
pca = PCA(n_components=2)
pca.fit(digits.data)
# transform the digits data onto the first two principal components
digits_pca = pca.transform(digits.data)
colors = ["#476A2A", "#7851B8", "#BD3430", "#4A2D4E", "#875525",
          "#A83683", "#4E655E", "#853541", "#3A3120","#535D8E"]
plt.figure(figsize=(10, 10))
plt.xlim(digits_pca[:, 0].min(), digits_pca[:, 0].max())
plt.ylim(digits_pca[:, 1].min(), digits_pca[:, 1].max())
for i in range(len(digits.data)):
    # actually plot the digits as text instead of using scatter
    plt.text(digits_pca[i, 0], digits_pca[i, 1], str(digits.target[i]),
             color = colors[digits.target[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("first principal component")
plt.ylabel("second principal component");

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42)
# use fit_transform instead of fit, as TSNE has no transform method:
digits_tsne = tsne.fit_transform(digits.data)

In [ ]:
plt.figure(figsize=(10, 10))
plt.xlim(digits_tsne[:, 0].min(), digits_tsne[:, 0].max() + 1)
plt.ylim(digits_tsne[:, 1].min(), digits_tsne[:, 1].max() + 1)
for i in range(len(digits.data)):
    # actually plot the digits as text instead of using scatter
    plt.text(digits_tsne[i, 0], digits_tsne[i, 1], str(digits.target[i]),
             color = colors[digits.target[i]],
             fontdict={'weight': 'bold', 'size': 9})

## Independent Component Analysis

May be a better choice than PCA for non-Gaussian distributions.

From: http://scikit-learn.org/stable/auto_examples/decomposition/plot_ica_vs_pca.html

In [ ]:
from sklearn import decomposition
def plotTwoCol(data, ax, *args, **kwargs):
    ax.plot(data[:,0], data[:,1], *args, **kwargs)

### Using a `t` distribution with small degrees of freedom (non-Gaussian)

In [ ]:
src = np.random.standard_t(1.5, size=(1000,2))
src[:, 0] *= 2

# create observed data as mixture of src
mix = np.array([[1, 1],   
                [0, 2]])
obs = np.dot(src, mix.T)

In [ ]:
# obs[i] = <src[i]*A[0], src[i]*A[1]>
print(np.dot(src[0], mix[0,:]), np.dot(src[0], mix[1,:]))
print(src[0], "-->", obs[0])

In [ ]:
obs /= np.std(obs, axis=0)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,5), sharey=True)
plotTwoCol(src/np.std(src,axis=0), ax1, "b.")
plotTwoCol(obs, ax2, 'r.')

ax1.set_title("True Sources")
ax2.set_title("Observed Data")

ax1.set_ylim(-4, 4)
ax1.set_xlim(-4, 4)
ax2.set_xlim(-4, 4)

In [ ]:
pca = decomposition.PCA()
rec_pca = pca.fit_transform(obs) # .transform(X)

ica = decomposition.FastICA()
rec_ica = ica.fit_transform(obs) # .transform(X)  # Estimate the sources

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,5), sharey=True)
plotTwoCol(rec_pca / np.std(rec_pca, axis=0), ax1, 'b.')
plotTwoCol(rec_ica / np.std(rec_ica, axis=0), ax2, 'r.')

ax1.set_title("Recovered via PCA")
ax2.set_title("Recovered via ICA")

ax1.set_ylim(-4,4)
ax1.set_xlim(-4,4)
ax2.set_xlim(-4,4)

In [ ]:
fig, axes =  plt.subplots(2, 2, figsize=(10,10), sharey=True, sharex=True)
plotTwoCol(src/np.std(src,axis=0), axes[0,0], "b.")
plotTwoCol(obs, axes[0,1], 'r.')

axes[0,0].set_title("True Sources")
axes[0,1].set_title("Observed Data")

plotTwoCol(rec_pca / np.std(rec_pca, axis=0), axes[1,0], 'g.')
plotTwoCol(rec_ica / np.std(rec_ica, axis=0), axes[1,1], 'y.')

axes[1,0].set_title("Recovered via PCA")
axes[1,1].set_title("Recovered via ICA")

for ax in axes.flat:
    ax.set_xlim(-4,4)
    ax.set_ylim(-4,4)

# Summary

In this notebook, we reviewed the following topics in preparation for more advanced topics:

 * [Dimensionality Reduction, Feature Extraction and Manifold Learning](#Dimensionality-Reduction,-Feature-Extraction-and-Manifold-Learning)
 * [Principal Component Analysis (PCA)](#Principal-Component-Analysis-%28PCA%29)
 * [Non-Negative Matrix Factorization (NMF)](#Non-Negative-Matrix-Factorization-%28NMF%29)
 * [Manifold learning with t-SNE](#Manifold-learning-with-t-SNE)
 * [Independent Component Analysis](#Independent-Component-Analysis)
 * [Exercises](#Exercises)
 * [Summary](#Summary)


<a href='Unsupervised_Feature_Extraction_Exercises.ipynb' class='btn btn-primary btn-lg'>Exercises</a>

<img src='img/copyright.png'>